In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV, LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np
import joblib

In [2]:
data = pd.read_parquet('./input/creditos_hist.parquet')

In [3]:
# Eliminamos las situaciones 0, que indican que el crédito ya fue pagado
data = data.loc[data['situacion'] != 0]
data = data.drop('denominacion', axis = 1) # Elimino la columna con las razones sociales para ahorrar RAM

In [4]:
# Una variable que puede ser de interés es cuantos créditos tiene una empresa en un momento dado del tiempo
counts = data.groupby(['identificacion', 'periodo']).size().reset_index(name='n_creditos')

# También nos interesa cuanta plata debe una empresa en cada momento dado
sums = data.groupby(['identificacion', 'periodo'], as_index=True)['monto'].sum().reset_index(name='sum_montos')

# La literatura indica que también importa la duración de la relación empresa-banco, por lo que contamos la cantidad 
# de periodos que aparece cada par: empresa-banco
period_counts = data.groupby(['identificacion', 'entidad']).size().reset_index(name='n_periodos')

# Definimos como default cuando el crédito se encuentra en situación 4 o 5, por lo que creamos la dummy de default
# Esta es nuestra variable dependiente
data['default'] = (data['situacion'] >= 4).astype(int)

In [5]:
# Queremos predecir el default el periodo siguienre
data['default_lag'] = data.groupby(['identificacion', 'entidad'])['default'].shift(1) # Lag a la variable default
data = data.dropna(subset=['default_lag']) # Eliminamos las observaciones que no tienen variable dependiente
data['default_lag'] = data['default_lag'].astype(int) # Cambio el dtype de la variable de interés

In [6]:
data = data.sort_values(by=['identificacion', 'periodo'])
data['prev_default'] = (
    data.groupby('identificacion')['default']
    .transform(lambda x: x.cumsum().clip(upper=1))
) # Armamos una variable que indique si en algún momento de su historia, esa empresa tuvo un crédito en default

In [7]:
# Agregamos las nuevas variables al dataframe
data = data.merge(counts, on=['identificacion', 'periodo'], how='left')
data = data.merge(sums, on=['identificacion', 'periodo'], how='left')
data = data.merge(period_counts, on=['identificacion', 'entidad'], how='left')

del sums, counts, period_counts # Para ahorrar RAM

In [8]:
data = data.loc[data['periodo'] > '202310']

In [9]:
# Elijo aleatoriamente un porcentaje de las empresas de la población
np.random.seed(42)
cuits = data['identificacion'].unique()
moneda = np.random.binomial(1, 0.05, len(cuits)) # Es como tirar una moneda sesgada para que agarre un porcentaje arbitrario de las empresas 
cuits_aleatorios = cuits[moneda == 1] # Estos son los cuits con los que me voy a quedar
data = data.loc[data['identificacion'].isin(cuits_aleatorios)] # Me quedo unicamente con las obs que tienen un cuit dentro de los seleccionados aleatoriamente

In [10]:
pv = pd.read_parquet('./input/principales_variables.parquet') # Datos de principales variables monetarias provenientes de la API del BCRA
pv.reset_index(inplace= True) # el index es la fecha, así que lo paso a columna
pv['fecha'] = pd.to_datetime(pv['fecha']) # paso la nueva columna al formato correcto
pv['periodo'] = pv['fecha'].dt.strftime('%Y%m') # armo una variable llamada periodo igual a la que tengo en los datos de la Central de Deudores
pv = pv.drop('fecha', axis = 1).groupby('periodo').agg(['mean', 'std']) # elimino la de "fecha" porque no me interesan los datos diarios
# Me quedo únicamente con los promedios por mes y también calculo el desvío estándar
pv.columns = ['_'.join(col).strip() for col in pv.columns] # Renombro las columnas para que sea más prolijo
pv.reset_index(inplace= True) # Vuelvo a agregar la columna periodo
pv = pv.loc[pv['periodo'].astype(int) <= 202411] # En la Central de Deudores tenemos datos hasta 202410
pv = pv.dropna(axis = 1) # Elimino las columnas con NAs

In [11]:
data = data.merge(pv, on = 'periodo', how = 'left') # Junto las principales variables monetarias con la Central de Deudores

In [12]:
emae = pd.read_excel('./input/sh_emae_mensual_base2004.xls', index_col=[0,1])
meses_a_numeros = {
    'Enero': '01', 'Febrero': '02', 'Marzo': '03', 'Abril': '04',
    'Mayo': '05', 'Junio': '06', 'Julio': '07', 'Agosto': '08',
    'Septiembre': '09', 'Octubre': '10', 'Noviembre': '11', 'Diciembre': '12'
}
emae = emae.reset_index()
emae['level_1'] = emae['level_1'].map(meses_a_numeros)
emae['periodo'] = emae['Período'].astype(str) + emae['level_1']
emae = emae.drop(columns=['level_1', 'Período'])

In [13]:
data = data.merge(emae, on = 'periodo', how= 'left')

In [14]:
arca = pd.read_parquet('./input/constancia_inscripcion.parquet') # Cargo los datos de la constancia de inscripción de ARCA
#arca = arca.drop(['direccion', 'localidad', 'razonSocial'], axis = 1) # Elimino algunas variables para ahorrar RAM

In [15]:
cuits_arca = set(arca['identificacion'])
cuits_bcra = set(data['identificacion'])

faltan = list(cuits_bcra - cuits_arca)

data['sin_arca'] = (data['identificacion'].isin(faltan)).astype(int)

In [16]:
#data = data.merge(arca, on = 'identificacion', how= 'left') # Junto las bases

In [17]:
# Por último, la literatura también resalta que la intensidad de la relación empresa-banco es relevante
# Usamos como proxy para la intensidad la proporción del monto adeudado con un banco sobre el total adeudado
data['monto_relativo'] = data['monto'] / data['sum_montos']

In [18]:
# Ponemos bien el tipo de dato para las columnas categóricas, así el get_dummies funciona bien
data['identificacion'] = data['identificacion'].astype('category')
data['entidad'] = data['entidad'].astype('category')
data['situacion'] = data['situacion'].astype('category')
data['default'] = data['default'].astype('category')
data['periodo'] = data['periodo'].astype('category')
#data['codPostal'] = data['codPostal'].astype('category')
#data['mesCierre'] = data['mesCierre'].astype('category')
#data['provincia'] = data['provincia'].astype('category')
data['default_lag'] = data['default_lag'].astype('category')
data['prev_default'] = data['prev_default'].astype('category')
data['sin_arca'] = data['sin_arca'].astype('category')

In [19]:
#Y = data.loc[data['IVA'].notna()]['default_lag'] # Nuestra variable de interés para las empresas para las cuales tenemos datos de ARCA

Para hacer cross validation, tenemos que tener en cuenta que tenemos un panel. Por lo tanto, vamos a entrenar el modelo con datos del pasado y evaluarlo con datos del futuro

In [20]:
# Cross Validation
data = data.sort_values(by='periodo') # Ordeno de acuerdo a la fecha
data = data.reset_index().drop(columns= 'index')
split_index = int(len(data) * 0.8) # El 80% de las observaciones más antiguas
train_indices = data.iloc[:split_index].index # Estos son los índices con los que después voy a separar en test y train
test_indices = data.iloc[split_index:].index

In [21]:
Y = data['default_lag']

In [22]:
boolean_columns = data.select_dtypes(include='object').columns # Estas son las columnas que ya están en el formato correcto
columnas = ['entidad', 'monto', 'n_creditos', 'sum_montos', 'n_periodos', 'monto_relativo', 'sin_arca', 'default', 'prev_default'] # Algunas de las variables independientes del modelo
#columnas.extend(arca.columns) # Todas las columnas de ARCA
pv.set_index('periodo', inplace= True)
columnas.extend(pv.columns)# Todas las columnas de las principales variables monetarias
emae.set_index('periodo', inplace= True)
columnas.extend(emae.columns) # Todas las columnas de emae

In [23]:
columns_to_encode = [col for col in columnas if col not in boolean_columns] # Una lista con las columnas que no tengo que meter en "get_dummies"
X_encoded = pd.get_dummies(data[columns_to_encode], drop_first=True) # Meto las columnas en get_dummies
X = pd.concat([X_encoded, data[boolean_columns]], axis=1) # Junto todas las variables independientes en un solo df

del columns_to_encode, X_encoded

In [24]:
#X = data[data['IVA'].notna()] # Las X para las que si tenemos datos de ARCA

In [25]:
# Intersección entre conjunto de entrenamiento y tienen datos para ARCA
#train_indices = [idx for idx in train_indices if idx in X.index]
#test_indices = [idx for idx in test_indices if idx in X.index]

In [26]:
# Separo en entrenamiento y test
X_train = X.loc[train_indices]
Y_train = Y.loc[train_indices]
X_test = X.loc[test_indices]
Y_test = Y.loc[test_indices]

También se puede hacer con neg_mean_squared_error, accuracy, f1_macro, f1_samples, average_precision, etc

## LASSO

In [29]:
tscv = TimeSeriesSplit(n_splits=10) # Este cross validation tiene en cuenta la temporarlidad de la base

pipeline = Pipeline([
    ('scaler', StandardScaler()), # Primero estandariza los datos
    ('logreg', LogisticRegressionCV( # Estima el modelo usando cross validation para elegir el mejor hiperparámetro
        cv=tscv,
        penalty='l1',
        solver='saga',
        scoring='accuracy',
        max_iter=2000,
        random_state= 42,
        tol = 1e-3,
        n_jobs= -1,
        fit_intercept= True,
        Cs=np.logspace(-4, 4, 20)
    ))
])

In [30]:
pipeline.fit(X_train, Y_train) # Entreno el modelo

Pipeline(steps=[('scaler', StandardScaler()),
                ('logreg',
                 LogisticRegressionCV(Cs=array([1.00000000e-04, 2.63665090e-04, 6.95192796e-04, 1.83298071e-03,
       4.83293024e-03, 1.27427499e-02, 3.35981829e-02, 8.85866790e-02,
       2.33572147e-01, 6.15848211e-01, 1.62377674e+00, 4.28133240e+00,
       1.12883789e+01, 2.97635144e+01, 7.84759970e+01, 2.06913808e+02,
       5.45559478e+02, 1.43844989e+03, 3.79269019e+03, 1.00000000e+04]),
                                      cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=10, test_size=None),
                                      max_iter=2000, n_jobs=-1, penalty='l1',
                                      random_state=42, scoring='accuracy',
                                      solver='saga', tol=0.001))])

In [47]:
joblib.dump(pipeline, './output/lasso_completo.pkl')

['./output/lasso_completo.pkl']

In [31]:
y_pred = pipeline.predict(X_test) # Hago las predicciones
cm = confusion_matrix(Y_test, y_pred) # Veo que tan bien funciona
print(cm)

[[37222    19]
 [  133  2527]]


Base: 									
|       | PN    | PP    |
|-------|-------|-------|	
| **TN** | 37226 | 15   |	
| **TP** | 2304  | 356  |

Base + prev_default:
|       | PN    | PP    |
|-------|-------|-------|	
| **TN** | 36542 | 699  |	
| **TP** | 297  | 2363  |			

Base + default:
|       | PN    | PP    |
|-------|-------|-------|	
| **TN** | 37222 | 19   |	
| **TP** | 133   | 2527 |

Base + sin_arca:
|       | PN    | PP    |
|-------|-------|-------|	
| **TN** | 37099 | 142  |	
| **TP** | 1692  | 968 	|

Completo:
|       | PN    | PP    |
|-------|-------|-------|	
| **TN** | 37222 | 19   |	
| **TP** | 133   | 2527 |


In [32]:
# Métricas
precision = precision_score(Y_test, y_pred)
recall = recall_score(Y_test, y_pred)
f1 = f1_score(Y_test, y_pred)
accuracy = accuracy_score(Y_test, y_pred)

print(f'La precisión es: {precision}')
print(f'El recall es: {recall}')
print(f'El f1 es: {f1}')
print(f'El accuracy es: {accuracy}')

La precisión es: 0.9925373134328358
El recall es: 0.95
El f1 es: 0.9708029197080292
El accuracy es: 0.9961905716648706


## Predictor Trivial
Si predecimos siempre negativo:
|       | PN    | PP    |
|-------|-------|-------|
| **TN** | 37241 | 0     |
| **TP** | 2660  | 0     |

* La precision sería: 0/0
* El recall sería: 0
* El f1 sería: 0
* El accuracy sería: 0.9333350041

In [33]:
model = pipeline.named_steps['logreg'] # Agarro el modelo desde el pipeline

In [34]:
best_c = model.C_
print(f'El mejor lambda para el modelo es: {1/best_c}')

El mejor lambda para el modelo es: [1438.44988829]


In [36]:
df_coeficientes = pd.DataFrame({
    'variable': X_train.columns,
    'coeficiente': model.coef_[0]
})
print(df_coeficientes)

                 variable  coeficiente
0                   monto     0.000000
1              n_creditos     0.000000
2              sum_montos     0.000000
3              n_periodos     0.000000
4          monto_relativo     0.000000
..                    ...          ...
217  entidad_Valerza S.A.     0.000000
218      entidad_YPF S.A.     0.000000
219            sin_arca_1     0.128646
220             default_1     1.591257
221        prev_default_1     0.255761

[222 rows x 2 columns]


## Elastic Net

In [48]:
tscv = TimeSeriesSplit(n_splits=10)

pipeline_en = Pipeline([
    ('scaler', StandardScaler()),  
    ('logreg', LogisticRegressionCV(
        cv=tscv,
        penalty='elasticnet',
        solver='saga',
        scoring='accuracy',
        max_iter=2000,
        random_state=42,
        tol=1e-3,
        n_jobs=-1,
        fit_intercept=True,
        Cs=np.logspace(-4, 4, 20), 
        l1_ratios=np.linspace(0.1, 0.9, 9)  
    ))
])

In [49]:
pipeline_en.fit(X_train, Y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('logreg',
                 LogisticRegressionCV(Cs=array([1.00000000e-04, 2.63665090e-04, 6.95192796e-04, 1.83298071e-03,
       4.83293024e-03, 1.27427499e-02, 3.35981829e-02, 8.85866790e-02,
       2.33572147e-01, 6.15848211e-01, 1.62377674e+00, 4.28133240e+00,
       1.12883789e+01, 2.97635144e+01, 7.84759970e+01, 2.06913808e+02,
       5.45559478e+02, 1.43844989e+03, 3.79269019e+03, 1.00000000e+04]),
                                      cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=10, test_size=None),
                                      l1_ratios=array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                                      max_iter=2000, n_jobs=-1,
                                      penalty='elasticnet', random_state=42,
                                      scoring='accuracy', solver='saga',
                                      tol=0.001))])

In [64]:
joblib.dump(pipeline_en, './output/elasticnet_completo.pkl')

['./output/elasticnet_completo.pkl']

In [50]:
y_pred = pipeline_en.predict(X_test) # Hago las predicciones
cm = confusion_matrix(Y_test, y_pred) # Veo que tan bien funciona
print(cm)

[[37222    19]
 [  133  2527]]


In [52]:
elasticnet = pipeline_en.named_steps['logreg']
elasticnet.l1_ratio_

array([0.2])

In [54]:
best_c = elasticnet.C_
print(f'El mejor lambda para el modelo es: {1/best_c}')

El mejor lambda para el modelo es: [1438.44988829]


In [55]:
df_coeficientes = pd.DataFrame({
    'variable': X_train.columns,
    'coeficiente': elasticnet.coef_[0]
})
print(df_coeficientes)

                 variable  coeficiente
0                   monto     0.000000
1              n_creditos     0.000000
2              sum_montos     0.000000
3              n_periodos     0.000000
4          monto_relativo     0.013745
..                    ...          ...
217  entidad_Valerza S.A.     0.000000
218      entidad_YPF S.A.     0.000000
219            sin_arca_1     0.359884
220             default_1     1.230199
221        prev_default_1     0.589436

[222 rows x 2 columns]


## RandomForest

In [27]:
tscv = TimeSeriesSplit(n_splits=10)

pipeline_rf = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestClassifier(
        n_estimators=100, 
        max_depth=None,   
        random_state=42,  
        n_jobs=-1,
        oob_score = True,          
    ))
])

In [28]:
pipeline_rf.fit(X_train, Y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('rf',
                 RandomForestClassifier(n_jobs=-1, oob_score=True,
                                        random_state=42))])

In [31]:
joblib.dump(pipeline_rf, './output/rf_completo.pkl')

['./output/rf_completo.pkl']

In [29]:
y_pred = pipeline_rf.predict(X_test) # Hago las predicciones
cm = confusion_matrix(Y_test, y_pred) # Veo que tan bien funciona
print(cm)

[[37223    18]
 [  133  2527]]


In [30]:
# Métricas
precision = precision_score(Y_test, y_pred)
recall = recall_score(Y_test, y_pred)
f1 = f1_score(Y_test, y_pred)
accuracy = accuracy_score(Y_test, y_pred)

print(f'La precisión es: {precision}')
print(f'El recall es: {recall}')
print(f'El f1 es: {f1}')
print(f'El accuracy es: {accuracy}')

La precisión es: 0.9929273084479371
El recall es: 0.95
El f1 es: 0.9709894332372718
El accuracy es: 0.9962156336933912


In [32]:
rf = pipeline_rf.named_steps['rf']
rf.oob_score_

0.9951755595097868

In [33]:
param_grid = {
    'rf__n_estimators': [100, 200, 400],  # Número de árboles
    'rf__max_depth': [None, 10, 20],  # Profundidad máxima de los árboles
}

grid_search = GridSearchCV(
    pipeline_rf, 
    param_grid, 
    cv=tscv, 
    scoring='accuracy', 
    n_jobs=-1
)

In [34]:
grid_search.fit(X_train, Y_train)

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=10, test_size=None),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('rf',
                                        RandomForestClassifier(n_jobs=-1,
                                                               oob_score=True,
                                                               random_state=42))]),
             n_jobs=-1,
             param_grid={'rf__max_depth': [None, 10, 20],
                         'rf__n_estimators': [100, 200, 400]},
             scoring='accuracy')

In [35]:
y_pred = grid_search.predict(X_test) # Hago las predicciones
cm = confusion_matrix(Y_test, y_pred) # Veo que tan bien funciona
print(cm)

[[37222    19]
 [  133  2527]]


## Modelo de Probabilidad Lineal

In [30]:
lm = LinearRegression(fit_intercept= True, n_jobs= -1)
lm.fit(X_train, Y_train)

LinearRegression(n_jobs=-1)

In [31]:
y_pred = lm.predict(X_test)
y_pred = np.where(y_pred >= 0.5, 1, 0)
cm = confusion_matrix(Y_test, y_pred)
print(cm)

[[ 5382 31859]
 [   19  2641]]


In [32]:
# Métricas
precision = precision_score(Y_test, y_pred)
recall = recall_score(Y_test, y_pred)
f1 = f1_score(Y_test, y_pred)
accuracy = accuracy_score(Y_test, y_pred)

print(f'La precisión es: {precision}')
print(f'El recall es: {recall}')
print(f'El f1 es: {f1}')
print(f'El accuracy es: {accuracy}')

La precisión es: 0.07655072463768116
El recall es: 0.9928571428571429
El f1 es: 0.1421420882669537
El accuracy es: 0.20107265482068118
